In [1]:
import os
import re
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
from dateutil.relativedelta import relativedelta
from sklearn import linear_model

In [2]:
df = pd.read_csv("final_dataset/final_v1_dataset.csv")
item_1_7_map = pd.read_csv("helper/ticker_item1_7_mapping.csv")

In [3]:
df.columns

Index(['Unnamed: 0', 'time', 'ticker', 'growth', 'y', 'volatility',
       'reaction_positive', 'reaction_negative', 'volume_adi', 'volume_mfi',
       'volatility_atr', 'trend_sma_slow', 'trend_macd', 'momentum_rsi',
       'momentum_roc', 'momentum_ppo', 'others_dlr', 'others_cr',
       'risk_factor_pos', 'risk_factor_neg', 'risk_factor_polarity',
       'risk_factor_subjectivity', 'risk_factor_len', 'item1_pos', 'item1_neg',
       'item1_polarity', 'item1_subjectivity', 'item1_len', 'item7_pos',
       'item7_neg', 'item7_polarity', 'item7_subjectivity', 'industry_B',
       'industry_C', 'industry_D', 'industry_E', 'industry_F', 'industry_G',
       'industry_H', 'industry_I'],
      dtype='object')

In [4]:
item_1_7_map.columns

Index(['Unnamed: 0', 'filename', 'firm', 'time', 'report_date'], dtype='object')

In [5]:
df.head()

,Unnamed: 0,time,ticker,growth,y,volatility,reaction_positive,reaction_negative,volume_adi,volume_mfi,...,item7_polarity,item7_subjectivity,industry_B,industry_C,industry_D,industry_E,industry_F,industry_G,industry_H,industry_I
0,0,2006,nwpx,1.175339,1,0.062822,5,6,-3.515938e+05,50.359009,...,0.519824,0.313428,0,0,1,0,0,0,0,0
1,1,2007,nwpx,0.709823,0,0.147784,5,6,-1.440436e+06,57.280553,...,0.487179,0.326236,0,0,1,0,0,0,0,0
2,2,2008,nwpx,0.566531,0,0.180493,3,8,-1.414003e+06,53.844115,...,0.490050,0.307105,0,0,1,0,0,0,0,0
3,3,2010,nwpx,1.331586,1,0.075963,10,1,-4.214262e+06,50.947005,...,0.429119,0.321627,0,0,1,0,0,0,0,0
4,4,2011,nwpx,1.252679,1,0.161627,6,5,-4.860406e+06,49.961210,...,0.500000,0.309386,0,0,1,0,0,0,0,0


In [6]:
# Readd 10K report date.
def add_report_date(df, item_1_7_map):
    report_date_ls = []
    for i in range(len(df)):
        cur_ticker = df['ticker'].iloc[i]
        cur_time = int(df['time'].iloc[i])
        cur_report_date = item_1_7_map[(item_1_7_map['firm'] == cur_ticker)
                                      & (item_1_7_map['time'] == cur_time)]['report_date'].iloc[0]
        report_date_ls.append(cur_report_date)
    df['report_date'] = report_date_ls
    return df

In [7]:
df = add_report_date(df, item_1_7_map)

In [8]:
t = str(df['report_date'].iloc[0])
t = dt.datetime.strptime(t, "%Y%m%d")
t

datetime.datetime(2007, 4, 2, 0, 0)

In [6]:
# count unique tickers
num_ticker = len(set(df['ticker']))
print("number of tickers are ", num_ticker)

number of tickers are  467


In [30]:
dic = {}

In [31]:
# Find number of companies for each industry
# industry = "B", 'C', 'D', 'E', 'F', 'G', 'H', 'I'
# industry = 'A' is the leftover
def find_num_comp(df, industry = "B", dic = dic):
    
    name = "industry_"  + industry
    temp_df = df[df[name] == 1]
    comp_count = len(temp_df['ticker'].unique())
    dic[industry] = comp_count
    return comp_count

In [32]:
num_A = num_ticker
industry = ['B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
for i in industry:
    cur_num = find_num_comp(df, i)
    num_A = num_A - cur_num
dic['A'] = num_A

In [33]:
dic

{'B': 7, 'C': 8, 'D': 216, 'E': 29, 'F': 14, 'G': 26, 'H': 90, 'I': 77, 'A': 0}

# Construct abnormal return

In [8]:
# We set estimation period one year before (20 days before event), considering the leakage effect
# We set the event window to be two years after first month of the 10-K report report date
# Since we think the first month after the report date may show some overestimation or underestimation
def event_study(data_ret, eventdate, ticker, estimation_period = 252, before_event = 20,
               event_window_start = 252, event_window_end = 756, benchmark = '^GSPC'):
    data_ret['post_event'] = (data_ret['Date'] >= eventdate).astype(int)
    data_ret = (data_ret.reset_index())
    
    event_date_index = data_ret.groupby(['post_event'])['index'].transform("min").max()
    data_ret['event_date_index'] = event_date_index
    
    data_ret['rel_day'] = data_ret['index'] - data_ret['event_date_index']
    
    estimation = data_ret[(data_ret['rel_day'] < -before_event) & (data_ret['rel_day'] >= -estimation_period - before_event)]
    event = data_ret[
        (data_ret['rel_day'] <= event_window_end) & (data_ret['rel_day'] >= event_window_start)
    ]
    
    x_df = estimation[benchmark].values.reshape(-1, 1)
    
    betas = []
    for y in [benchmark, ticker]:
        y_df = estimation[y].values.reshape(-1, 1)
        reg = linear_model.LinearRegression()
        betas.append(reg.fit(x_df, y_df).coef_)
        
    beta_np = np.array(betas)
    
    # Expected Returns via Beta
    # Need Numpy Array to do Calculations!
    sp500array = event[benchmark].values
    expected_returns = np.outer(sp500array, beta_np)
    expected_returns = pd.DataFrame(expected_returns, index=event.index)
    expected_returns.columns = [benchmark, ticker]
    expected_returns = expected_returns.rename(columns={ticker: "expected_return"})
    del expected_returns[benchmark]
    
    event = pd.concat([event, expected_returns], axis=1, ignore_index=False)

    event["abnormal_return"] = event[ticker] - event["expected_return"]
    
    car = sum(event['abnormal_return'])
    
    return car

In [9]:
for i, r in df.iterrows():
    t = str(r['report_date'])
    symbol = r['ticker'].upper()
    symbols_list = ['^GSPC', symbol]
    t = dt.datetime.strptime(t, '%Y%m%d')
    start = (t - relativedelta(years = 1)).strftime("%Y-%m-%d")
    end = (t+ relativedelta(years=3)).strftime("%Y-%m-%d")
    data = yf.download(symbols_list, start=start, end=end)

    # Calculate returns
    main_data = data["Adj Close"] / data["Adj Close"].shift(1) - 1
    main_data = main_data.dropna()
    main_data = main_data.reset_index()
    current_car = event_study(main_data, ticker = symbol, eventdate = t)
    df.loc[i, "abnormal_return"] = current_car
df.to_csv("final_dataset/car_one_year_after.csv")

[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%********

[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%********

[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%********

[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%********

[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%********

...

[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%********

[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%********

[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%********

[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%********

[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%********

In [39]:
df.head()

,Unnamed: 0,time,ticker,growth,y,volatility,reaction_positive,reaction_negative,volume_adi,volume_mfi,...,industry_B,industry_C,industry_D,industry_E,industry_F,industry_G,industry_H,industry_I,report_date,abnormal_return
0,0,2006,nwpx,1.175339,1,0.062822,5,6,-3.515938e+05,50.359009,...,0,0,1,0,0,0,0,0,20070402,0.378175
1,1,2007,nwpx,0.709823,0,0.147784,5,6,-1.440436e+06,57.280553,...,0,0,1,0,0,0,0,0,20080317,0.575797
2,2,2008,nwpx,0.566531,0,0.180493,3,8,-1.414003e+06,53.844115,...,0,0,1,0,0,0,0,0,20090313,-0.564693
3,3,2010,nwpx,1.331586,1,0.075963,10,1,-4.214262e+06,50.947005,...,0,0,1,0,0,0,0,0,20110322,-0.004541
4,4,2011,nwpx,1.252679,1,0.161627,6,5,-4.860406e+06,49.961210,...,0,0,1,0,0,0,0,0,20120427,-0.063412


In [42]:
df.to_csv("final_dataset/temp_data.csv")